### First thing first -- setup the grid world



In [6]:
import numpy as np

class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
        
    def set(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions
        
    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]
        
    def current_state(self):
        return (self.i, self.j)
    
    def is_terminal(self, s):
        return s not in self.actions
    
    def move(self, action):
        
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
            
        return self.rewards.get((self.i, self.j), 0)
    
    def undo_move(self, action):
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
            
        
        assert(self.current_state() in self.all_states())
        
    
    def game_over(self):
        
        return (self.i, self.j) not in self.actions
        
    def all_states(self):
        
        return set(list(self.actions.keys()) + list(self.rewards.keys()))
    
    
    def standard_grid():
        
        g = Grid(3,4,(2,0))
        rewards = {(0,3):1, (1,3): -1}
        actions = {
            (0, 0): ('D', 'R'),
            (0, 1): ('L', 'R'),
            (0, 2): ('L', 'D', 'R'),
            (1, 0): ('U', 'D'),
            (1, 2): ('U', 'D', 'R'),
            (2, 0): ('U', 'R'),
            (2, 1): ('L', 'R'),
            (2, 2): ('L', 'R', 'U'),
            (2, 3): ('L', 'U'),
        }
        
        g.set(rewards, actions)
        return g
    
    def negative_grid(step_cost=-0.1):
        g = Grid.standard_grid()
        g.rewards.update({
            (0, 0): step_cost,
            (0, 1): step_cost,
            (0, 2): step_cost,
            (1, 0): step_cost,
            (1, 2): step_cost,
            (2, 0): step_cost,
            (2, 1): step_cost,
            (2, 2): step_cost,
            (2, 3): step_cost,
            
        })
        
        return g
    

### Define some helper function to show current status

In [7]:
SMALL_ENOUGH = 1e-3

def print_values(V, g):
    
    print("width:%d, height:%d" % (g.width, g.height))
    
    for i in range(g.width):
        print("------------------------")
        
        for j in range(g.height):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        
        print ("")
        
def print_policy(P, g):
    for i in range(g.width):
        print("---------------------------")
        for j in range(g.height):
            a = P.get((i,j), 0)
            print(" %s |" % a, end="")
        print ("")



### Now we are ready to implement Bellman Equation, using iterative policy evaluation

Finding V(S) for a given policy (uniform random action in this case)

In [8]:

grid = Grid.standard_grid()

states = grid.all_states()
V = {}
print_values(V, grid)

for s in states:
    V[s] = 0

gamma = 1.0 # discount factor

#repeat while converge
while True:
    biggest_change = 0
    for s in states:
        
        old_v = V[s]
        
        if s in grid.actions:
            
            new_v = 0
            p_a = 1.0 / len(grid.actions[s])
            
            
            
            for a in grid.actions[s]:
                grid.set_state(s)
                r = grid.move(a)
                new_v += p_a * ( r + gamma * V[grid.current_state()])
                
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
            
    if biggest_change < SMALL_ENOUGH:
        print("Converged !")
        break
        
    print("values for uniformly random actions")
    print_values(V, grid)
    print("Finished")


width:3, height:4
------------------------
 0.00| 0.00| 0.00| 0.00|
------------------------
 0.00| 0.00| 0.00| 0.00|
------------------------
 0.00| 0.00| 0.00| 0.00|
values for uniformly random actions
width:3, height:4
------------------------
 0.00| 0.00| 0.22| 0.00|
------------------------
 0.00| 0.00|-0.33| 0.00|
------------------------
 0.00| 0.00|-0.28|-0.50|
Finished
values for uniformly random actions
width:3, height:4
------------------------
 0.06| 0.11| 0.25| 0.00|
------------------------
-0.01| 0.00|-0.35| 0.00|
------------------------
-0.07|-0.14|-0.38|-0.64|
Finished
values for uniformly random actions
width:3, height:4
------------------------
 0.07| 0.15| 0.26| 0.00|
------------------------
-0.02| 0.00|-0.37| 0.00|
------------------------
-0.11|-0.22|-0.43|-0.69|
Finished
values for uniformly random actions
width:3, height:4
------------------------
 0.07| 0.17| 0.26| 0.00|
------------------------
-0.04| 0.00|-0.39| 0.00|
------------------------
-0.15|-0.27|-0

### Now let's look at control problem - How to find a better policy ( and then the optimal policy)

### the process will finish when the value function no-longer change. No need to check value function converge. Because once policy become constant so will value



In [9]:

ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
SMALL_ENOUGH = 1e-3


grid = Grid.negative_grid()
states = grid.all_states()
V = {}
policy = {}


print("Rewards:")
print_values(grid.rewards, grid)

for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

print("Initial Policy:")
print_policy(policy, grid)

for s in states:
    if s in grid.actions:
        V[s] = np.random.random()
    else:
        V[s] = 0

print("Initial State Value:")
print_values(V, grid)        
        
        
gamma = 0.9 # discount factor


#repeat while converge
while True :  
       
    ## POLICY EVALUATION
    
    while True:
        biggest_change = 0
        for s in states:
            
            old_v = V[s]

            if s in policy:

                a = policy[s]
                grid.set_state(s)
                r = grid.move(a)
                V[s] =  r + gamma * V[grid.current_state()]
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        
        print("biggest change: %f" % biggest_change)
        if biggest_change < SMALL_ENOUGH:
            break

        print("values for uniformly random actions")
        print_values(V, grid)
        print("")

    
    ## POLICY IMPROVEMENT STEP

    is_policy_converged = True
    for s in states:
        for s in policy:

            old_a = policy[s]
            new_a = None
            best_value = float('-inf')

            for a in ALL_POSSIBLE_ACTIONS:

                grid.set_state(s)
                r = grid.move(a)
                v = r + gamma * V[grid.current_state()]

                if v > best_value:
                    best_value = v
                    new_a = a

            policy[s] = new_a

            if new_a != old_a:
                is_policy_converged = False
    print("updated policy:")
    print_policy(policy, grid)             
    if is_policy_converged:  
       
        break
            
print("final values:")
print_values(V, grid)
print("final policy:")
print_policy(policy, grid)

                    

Rewards:
width:3, height:4
------------------------
-0.10|-0.10|-0.10| 1.00|
------------------------
-0.10| 0.00|-0.10|-1.00|
------------------------
-0.10|-0.10|-0.10|-0.10|
Initial Policy:
---------------------------
 L | D | D | 0 |
---------------------------
 L | 0 | U | 0 |
---------------------------
 L | L | U | D |
Initial State Value:
width:3, height:4
------------------------
 0.74| 0.89| 0.70| 0.00|
------------------------
 0.94| 0.00| 0.47| 0.00|
------------------------
 0.22| 0.15| 0.91| 0.62|
biggest change: 0.535867
values for uniformly random actions
width:3, height:4
------------------------
 0.56| 0.70| 0.38| 0.00|
------------------------
 0.75| 0.00| 0.53| 0.00|
------------------------
 0.10| 0.10| 0.38| 0.46|

biggest change: 0.290856
values for uniformly random actions
width:3, height:4
------------------------
 0.41| 0.53| 0.12| 0.00|
------------------------
 0.58| 0.00| 0.24| 0.00|
------------------------
-0.01|-0.01| 0.12| 0.31|

biggest change: 0.23559

### Policy Iteration is not very efficient, because we have to wait V(s) to converge before we can improve policy

### Value Iteration try to merge policy evaluation and improvement into one step. No need to wait for value function converge. 

In [10]:
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
SMALL_ENOUGH = 1e-4


grid = Grid.negative_grid()
states = grid.all_states()
V = {}
policy = {}


print("Rewards:")
print_values(grid.rewards, grid)

for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

print("Initial Policy:")
print_policy(policy, grid)

for s in states:
    if s in grid.actions:
        V[s] = np.random.random()
    else:
        V[s] = 0

print("Initial State Value:")
print_values(V, grid)        
        
        
gamma = 0.9 # discount factor



while True:
    biggest_change = 0
    for s in states:

        old_v = V[s]

        if s in policy:

            a = policy[s]
            grid.set_state(s)
            r = grid.move(a)
            V[s] =  r + gamma * V[grid.current_state()]
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    print("biggest change: %f" % biggest_change)
    if biggest_change < SMALL_ENOUGH:
        break

    print("values for uniformly random actions")
    print_values(V, grid)
    print("")

    ## NEW - MERGE INTO LOOP
    for s in policy.keys():

        best_a = None
        best_value = float('-inf')


        for a in ALL_POSSIBLE_ACTIONS:

            grid.set_state(s)
            r = grid.move(a)
            v = r + gamma * V[grid.current_state()]

            if v > best_value:
                best_value = v
                best_a = a

        policy[s] = best_a

    print("updated policy:")
    print_policy(policy, grid)          

print("final values:")
print_values(V, grid)
print("final policy:")
print_policy(policy, grid)

Rewards:
width:3, height:4
------------------------
-0.10|-0.10|-0.10| 1.00|
------------------------
-0.10| 0.00|-0.10|-1.00|
------------------------
-0.10|-0.10|-0.10|-0.10|
Initial Policy:
---------------------------
 U | D | L | 0 |
---------------------------
 D | 0 | D | 0 |
---------------------------
 U | U | D | U |
Initial State Value:
width:3, height:4
------------------------
 0.21| 0.72| 0.14| 0.00|
------------------------
 0.45| 0.00| 0.80| 0.00|
------------------------
 0.74| 0.56| 0.81| 0.16|
biggest change: 1.162033
values for uniformly random actions
width:3, height:4
------------------------
 0.09| 0.54| 0.39| 0.00|
------------------------
 0.17| 0.00| 0.63| 0.00|
------------------------
 0.30| 0.41| 0.63|-1.00|

updated policy:
---------------------------
 R | U | R | 0 |
---------------------------
 D | 0 | D | 0 |
---------------------------
 R | R | U | L |
biggest change: 1.469105
values for uniformly random actions
width:3, height:4
-----------------------